In [1]:
%run "../Get Features.ipynb"

In [2]:
import numpy as np

In [3]:
%%time
a = get_all()
a = [np.hstack(i) for i in a]
pronoun, verbs, entity_counts, sen_len, sen_pos, stop_counts, output = a

loaded tags
loaded doc metadata
loaded stop counts
loaded outputs
Wall time: 4.22 s


In [4]:
output = [0 if x == 2 else x for x in output]

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

In [6]:
from model import LogisticRegression

In [7]:
input = torch.t(torch.FloatTensor(a[:-1]))
output = torch.Tensor(output).unsqueeze(1)

In [8]:
model = LogisticRegression()

In [9]:
LOSS = nn.BCEWithLogitsLoss()
OPTIM = optim.Adam(model.parameters())

In [10]:
NO_ITERATIONS = 10
BATCH_SIZE = 20

In [11]:
LOSS(model(input), output)

tensor(3.0501, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [12]:
for i in range(NO_ITERATIONS):
    for j in range(0, len(input), BATCH_SIZE):
        model_output = model(input[j: j+BATCH_SIZE])
        truth = output[j: j+BATCH_SIZE]
        loss = LOSS(model_output, truth)
        model.zero_grad()
        loss.backward()
        OPTIM.step()
        print(j, end='\r')

In [13]:
LOSS(model(input), output)

tensor(0.5894, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

In [14]:
model.weights.weight

Parameter containing:
tensor([[-0.0144, -0.0044,  0.2282,  0.0741, -0.0292, -0.0797]],
       requires_grad=True)

In [15]:
torch.save(model.state_dict(), 'log_reg_weights')